In [ ]:
# Vectors, matrices, and multidimensional arrays
# Network basics
# Activation function
# Fully connected network realized from scratch

In [18]:
import numpy as np
# Create row vector
row_vector = np.array([1, 2, 3, 4, 5])
print("Row vector: ", row_vector)

# Create column vector
column_vector = np.array([[1], [2], [3], [4], [5]])
print("Column vector: ", column_vector)

# Create matrix
matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print("Matrix: ", matrix)

Row vector:  [1 2 3 4 5]
Column vector:  [[1]
 [2]
 [3]
 [4]
 [5]]
Matrix:  [[1 2 3]
 [4 5 6]
 [7 8 9]]


In [ ]:
import numpy as np
# transpose matrix
matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print("Matrix: ")
print(matrix)
print("Transpose matrix: ")
print(matrix.T)
print(np.transpose(matrix))

# inverse matrix
matrix = np.array([[1, 2], [3, 4]])
print("Matrix: ")
print(matrix)
print("Inverse Matrix: ")
print(np.linalg.inv(matrix))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 1. Define the neural network model
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)  # First fully connected layer (input: 784, output: 512)
        self.fc2 = nn.Linear(512, 256)      # Second fully connected layer (input: 512, output: 256)
        self.fc3 = nn.Linear(256, 128)      # Third fully connected layer (input: 256, output: 128)
        self.fc4 = nn.Linear(128, 10)       # Output layer (input: 128, output: 10 classes)

    def forward(self, x):
        x = x.view(-1, 28 * 28)             # Flatten the input tensor
        x = torch.relu(self.fc1(x))         # Apply ReLU activation function
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)                     # No activation function at the output layer
        return x

# 2. Load and preprocess the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),                 # Convert images to tensor
    transforms.Normalize((0.5,), (0.5,))   # Normalize the images
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 3. Initialize the network, loss function, and optimizer
model = FCNN()
criterion = nn.CrossEntropyLoss()           # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# 4. Train the network
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()           # Zero the parameter gradients
            outputs = model(images)         # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()                 # Backward pass (compute gradients)
            optimizer.step()                # Update weights
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}')

# 5. Evaluate the network
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct / total:.2f}%')

# Run the training and evaluation
train(model, train_loader, criterion, optimizer)
evaluate(model, test_loader)
